# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

Please make sure if you are running this notebook in the workspace that you have chosen GPU rather than CPU mode.

In [1]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import torch
#import helper
from torch import nn
import torch.nn.functional as F
from torch import optim
from torchvision import datasets, transforms
import torchvision.models as models
import numpy as np

from PIL import Image

#import argparse

from collections import OrderedDict

from workspace_utils import active_session

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
test_dir = data_dir + '/test'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [3]:
# TODO: Define your transforms for the training, validation, and testing sets
# For the training, you'll want to apply transformations such as random scaling, cropping, and flipping.
# input data is resized to 224x224 pixels
# For validation and testing sets, no transformations, but you'll need to resize then crop the images to the appropriate size
# normalize the means and standard deviations of the images
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(), 
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
data_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(), 
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
#class torchvision.datasets.ImageFolder(root, transform=None, target_transform=None, loader=<function default_loader>)
trainset = datasets.ImageFolder('flowers/train/', transform = train_transforms)
testset = datasets.ImageFolder(test_dir, transform = data_transforms)
validset = datasets.ImageFolder(valid_dir, transform = data_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
#dataloaders = 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=32, shuffle=True)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

In [5]:
#load a pretrained network
model = models.vgg16(pretrained= True)
model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:04<00:00, 122754689.44it/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [6]:
#input (from vgg16): 25088, hidden layer: 4096, output: 1000, dropout 0.5
# TODO: Build and train your network

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Hyperparameters for the network
input_size = 25088
#hidden_sizes = [4096, 1024]
hidden_sizes = 1024
output_size = 102


# Build a feed-forward network
classifier = nn.Sequential(OrderedDict([
                                        ('fc1', nn.Linear(input_size, hidden_sizes)),
                                        ('dropout1', nn.Dropout(0.2)),
                                        ('relu1', nn.ReLU()),
#                                        ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
#                                        ('dropout2', nn.Dropout(0.5)),
#                                        ('relu2', nn.ReLU()),
                                        ('logits', nn.Linear(hidden_sizes, output_size)),
                                        ('output', nn.LogSoftmax(dim=1))
                                        ]))
    
model.classifier = classifier
print(model)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [7]:
#train your network
with active_session():
    model.to(device)

    criterion = nn.NLLLoss()
    #optimizer = optim.SGD(model.parameters(), lr=0.01)
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
    epochs = 5 #update epochs to 3-5 later
    print_every = 40
    steps = 0

    for e in range(epochs):
        model.classifier.train()
    #    model.train()
        train_loss = 0
        train_accuracy = 0
        steps = 0
        float_tensor = torch.FloatTensor if device == 'cpu' else torch.cuda.FloatTensor

        for images, labels in iter(trainloader):
            steps += 1

            optimizer.zero_grad()

            images, labels = images.to(device), labels.to(device)
            # Forward and backward passes
     #       output = model.classifier.forward(images)
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

     #TODO add accuracy calculation here######
            ps = torch.exp(output)
            equality = (labels.data == ps.max(dim=1)[1])
            train_accuracy += equality.type(float_tensor).mean()
    #####
            if steps % print_every == 0:
                print("Epoch: {}/{}... ".format(e+1, epochs),
                      "Train Loss: {:.4f}".format(train_loss/print_every),
                      "Train Accuracy: {:.4f}".format(train_accuracy/print_every))
     #TODO print accuracy here########

                train_loss = 0

Epoch: 1/5...  Train Loss: 3.3762 Train Accuracy: 0.3039
Epoch: 1/5...  Train Loss: 1.6270 Train Accuracy: 0.8883
Epoch: 2/5...  Train Loss: 0.9720 Train Accuracy: 0.7328
Epoch: 2/5...  Train Loss: 0.9383 Train Accuracy: 1.4809
Epoch: 3/5...  Train Loss: 0.8204 Train Accuracy: 0.7730
Epoch: 3/5...  Train Loss: 0.7543 Train Accuracy: 1.5684
Epoch: 4/5...  Train Loss: 0.7807 Train Accuracy: 0.7887
Epoch: 4/5...  Train Loss: 0.7134 Train Accuracy: 1.5852
Epoch: 5/5...  Train Loss: 0.6527 Train Accuracy: 0.8148
Epoch: 5/5...  Train Loss: 0.6857 Train Accuracy: 1.6266


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [8]:
# Implement a function for the test pass
def test(model, testloader, criterion, device):
#    print('in test function')
    test_loss = 0
    test_accuracy = 0
    
    float_tensor = torch.FloatTensor if device == 'cpu' else torch.cuda.FloatTensor
    
    for images, labels in testloader:
#        print('in test function, loading data')
        images, labels = images.to(device), labels.to(device)
#        output = model.classifier.forward(images)
        output = model.forward(images)
        test_loss += criterion(output, labels).item()

        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        test_accuracy += equality.type(float_tensor).mean()
    
    return test_loss, test_accuracy

In [9]:
# TODO: Do test on the test set
with active_session():
#print ('in test')
    model.to(device)
    e = 0
    epochs = 2
    steps = 0
    running_loss = 0
    print_every = 10 #40
    for e in range(epochs):
    #    model.eval()
    #    model.classifier.eval()
        for images, labels in testloader:
    #        print ('in test testloader for loop')
            steps += 1
            torch.no_grad()
            images, labels = images.to(device), labels.to(device)
    #        output = model.classifier.forward(images)
            output = model.forward(images)
            loss = criterion(output, labels)

            running_loss += loss.item()

            if steps % print_every == 0:
                # Make sure network is in eval mode for inference
                model.eval()
    #            print('before calling test function')
                # Turn off gradients for validation, saves memory and computations
                with torch.no_grad():
                    test_loss, accuracy = test(model, testloader, criterion, device)
                    loss.backward()
                    optimizer.step()

                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                      "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
                      "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

                model.train()
                running_loss = 0


Epoch: 1/2..  Training Loss: 0.757..  Test Loss: 0.501..  Test Accuracy: 0.875
Epoch: 1/2..  Training Loss: 0.503..  Test Loss: 0.472..  Test Accuracy: 0.871
Epoch: 2/2..  Training Loss: 0.643..  Test Loss: 0.512..  Test Accuracy: 0.873
Epoch: 2/2..  Training Loss: 0.876..  Test Loss: 0.669..  Test Accuracy: 0.827
Epoch: 2/2..  Training Loss: 1.154..  Test Loss: 1.072..  Test Accuracy: 0.760


In [10]:
# Implement a function for the validation pass
def validation(model, validloader, criterion, device):
#    print('in validation function')
   
    valid_loss = 0
    valid_accuracy = 0
    
    float_tensor = torch.FloatTensor if device == 'cpu' else torch.cuda.FloatTensor
        
    for images, labels in validloader:
#        print('in validation function, loading data')
        images, labels = images.to(device), labels.to(device)
#        output = model.classifier.forward(images)
        output = model.forward(images)
        valid_loss += criterion(output, labels).item()

        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        valid_accuracy += equality.type(float_tensor).mean()
    
    return valid_loss, valid_accuracy

In [11]:
# TODO: Do validation on the validation set
with active_session():
    #print ('in validation')
    model.to(device)
    e = 0
    epochs = 2
    steps = 0
    running_loss = 0
    print_every = 10 #40
    for e in range(epochs):
    #    model.eval()
    #    print ('in validation epoch')
    #    model.classifier.eval()
        for images, labels in validloader:
    #        print ('in validation validloader for loop')
            images, labels = images.to(device), labels.to(device)
            steps += 1
            torch.no_grad()
    #        output = model.classifier.forward(images)
            output = model.forward(images)
            loss = criterion(output, labels)

            running_loss += loss.item()

            if steps % print_every == 0:
                # Make sure network is in eval mode for inference
                model.eval()
     #           print('before calling validation function')
                # Turn off gradients for validation, saves memory and computations
                with torch.no_grad():
                    valid_loss, accuracy = validation(model, validloader, criterion, device)

                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Validation Loss: {:.3f}.. ".format(valid_loss/len(validloader)),
                      "Validation Accuracy: {:.3f}".format(accuracy/len(validloader)))

                model.train()            
                running_loss = 0


Epoch: 1/2..  Validation Loss: 1.820..  Validation Accuracy: 0.699
Epoch: 1/2..  Validation Loss: 1.829..  Validation Accuracy: 0.696
Epoch: 2/2..  Validation Loss: 1.838..  Validation Accuracy: 0.695
Epoch: 2/2..  Validation Loss: 1.842..  Validation Accuracy: 0.697
Epoch: 2/2..  Validation Loss: 1.852..  Validation Accuracy: 0.695


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [12]:
# TODO: Save the checkpoint 
with active_session():
    #model.class_to_idx = image_datasets['train'].class_to_idx
    model.class_to_idx = trainset.class_to_idx

    checkpoint = {'input_size': 25088,
                  'output_size': 102,
                  'hidden_layers': 1024,
                  'learning_rate': 0.001,
                  'classifier' : model.classifier,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict,
                  'epochs': 2,
    #              ```model.class_to_idx = image_datasets['train'].class_to_idx``` 
                  'class_to_idx': model.class_to_idx
                 }

    torch.save(checkpoint, 'checkpoint.pth')


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [13]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
#    checkpoint= torch.load(filepath, map_location=lambda storage, loc: storage) # load in CPU mode   
    model = load_checkpoint('checkpoint.pth')
    model = getattr(models, checkpoint['arch'])(pretrained=True)  
    model.classifier = checkpoint['classifier']
    model.class_to_idx = checkpoint['class_to_idx']
    #model.load_class_to_idx(checkpoint['class_to_idx'])
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
 
    print(model)
    return model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [14]:
def process_image(image):
#    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
#        returns an Numpy array
#    '''
    # TODO: Process a PIL image for use in a PyTorch model
#    image.resize_(256) 
    image.thumbnail(256)
    image.crop_(224)
    
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    pil_image = (image - mean) / std

    np_image = np.array(pil_image)
    
    np_image = np_image.transpose((1, 2, 0))
    return np_image

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [15]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [16]:
def predict(image_path, model, device, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    # TODO: Implement the code to predict the class from an image file  
    model.to(device)  
    model.eval()

    float_tensor = torch.FloatTensor if device == 'cpu' else torch.cuda.FloatTensor
    
    dataiter = iter(validloader)
    images, labels = dataiter.next()
    
    images, labels = images.to(device), labels.to(device)
    
    img = images[0]
# Convert 2D image to 1D vector
    img = img.view(1, 25088)
    
#process image here and display?
#    img = process_image(img)
#    imshow(img)
    
# Calculate the class probabilities (softmax) for img
    with torch.no_grad():
        output = model.forward(img)

    ps = torch.exp(output)
    topk_prob, topk_classes = torch.topk(ps, topk, sorted = True)
    topk_prob = topk_prob[0].cpu().numpy()
    topk_classes = topk_classes[0].cpu().numpy()
    
    print ('topk prob:', topk_prob)
    print ('topk classes:', topk_prob) 
#    inv_class_idx = {v: k for k,v in model.class_to_idx.items()}
#    print([prob.item() for prob in probs[0].data], [inv_class_idx[i.item()] for i in inds[0].data])
    
    return ps


## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
with active_session():
    image_path = test_dir + "/10/image_07104.jpg"
    model = load_checkpoint('checkpoint.pth')

    probs, classes = predict(image_path, model, "cuda:0", topk=5)
    print(probs)
    print(classes)

    #    cat_to_name = json.load(f)

    names = [cat_to_name[c] for c in classes]
    fig = plt.figure(figsize=(4, 7))
    ax1 = fig.add_subplot(2, 1, 1)
    ax1.xaxis.set_visible(False)
    ax1.yaxis.set_visible(False)
    imshow(process_image(image_path), ax=ax1, title=cat_to_name["10"])
    ax2 = fig.add_subplot(2, 1 , 2)
    ax2.barh(names, probs)